<h1><center>Лабораторна робота 4.</center></h1>
<h2><center>Виявлення сарказму за текстовими повідомленнями з використанням логістичної регресії</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

В цій лабораторній роботі ми будемо подувати модель логістичної регресії для розв'язання задачі класифікації саркастичних текстів. Для цього використаємо набір даних з [наукової статі](https://arxiv.org/abs/1704.05579) «A Large Self-Annotated Corpus for Sarcasm» з понад 1 мільйоном коментарів із платформи Reddit, що позначені авторами статті як саркастичні або ні. Оброблена версія цього набору даних міститься на платформі Kaggle у формі [Набору даних Kaggle](https://www.kaggle.com/datasets/danofer/sarcasm/data).

<a class="anchor" id="lab-4"></a>

## Зміст

- [4.1. Попередній аналіз даних](#lab-4.1)
- [4.2. Векторизація TF-IDF та логістична регресія](#lab-4.2)
- [4.3. Інтерпретація та порівняння моделі](#lab-4.3)
- [4.4. Розширене вдосконалення моделі](#lab-4.4)
- [4.5. Практичне застосування результатів інтелектуального аналізу даних](#lab-4.5)

Спершу завантажте цільовий набір даних `train-balanced-sarcasm.csv` із ресурсу Kaggle за [посиланням](https://www.kaggle.com/datasets/danofer/sarcasm/data?select=train-balanced-sarcasm.csv). Збережіть файл .csv в теку з робочим файлом лабораторної роботи.

Цей набір даних містить 1,3 млн. саркастичних коментарів з платформи інтернет-коментарів Reddit. Набір даних було створено шляхом вилучення коментарів з Reddit, що містять тег `\s` ("сарказм"). Цей тег переважно використовується користувачами, щоб вказати на жартівливий тон їхнього тексту, тобто такий запис не повинен сприйматися серйозно, і, як правило, є надійним показником саркастичного змісту запису.

Дані мають збалансовану та незбалансовану (тобто справжній розподіл) версії. Реальне співвідношення несаркастичних до саркастичних коментарів становить приблизно 1:100. За потреби оригінальний набір даних можна отримати за [посиланням](https://github.com/NLPrinceton/SARC).


In [1]:
PATH_TO_DATA = "train-balanced-sarcasm.csv"

In [2]:
# Виконаємо завантаження основних бібліотек
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
train_df = pd.read_csv(PATH_TO_DATA)

Нижче переглянемо зразки цільового набору в розрізі основних ознак.

In [4]:
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010826 non-null  int64 
 1   comment         1010773 non-null  object
 2   author          1010826 non-null  object
 3   subreddit       1010826 non-null  object
 4   score           1010826 non-null  int64 
 5   ups             1010826 non-null  int64 
 6   downs           1010826 non-null  int64 
 7   date            1010826 non-null  object
 8   created_utc     1010826 non-null  object
 9   parent_comment  1010826 non-null  object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


Деяких коментарів бракує в наборі, тому ми видалимо відповідні рядки.

In [6]:
train_df.dropna(subset=["comment"], inplace=True)

Переконаємось, що розглядуваний набір даних є справді збалансованим.

In [7]:
train_df["label"].value_counts()

0    505405
1    505368
Name: label, dtype: int64

Розділимо дані на частини для навчання та валідації.

In [8]:
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17
)

<a class="anchor" id="lab-4.1"></a>

## <span style="color:blue; font-size:1.2em;">4.1. Попередній аналіз даних</span>

[Повернутися до змісту](#lab-4)

In [9]:
# from wordcloud import STOPWORDS, WordCloud

In [10]:
# Підготуємо об'єкт класу WordCloud для 
# можливого візуального аналізу  
# my_wordcloud = WordCloud(
#     background_color="black",
#     stopwords=STOPWORDS,
#     max_words=200,
#     max_font_size=100,
#     random_state=17,
#     width=800,
#     height=400,
# )

<span style="color:red; font-size:2em;">Завдання 1</span>

---

**Варіант 1:** Проаналізуйте структуру текстових даних, включно з кількістю символів та слів у кожному повідомленні. Оцініть, як довжина тексту може вплинути на точність виявлення сарказму. Використайте для цього функції `len()` для обчислення кількості символів та метод `apply(lambda x: len(x.split()))` для обчислення кількості слів у датафреймі Pandas.

---

**Варіант 2:** Здійсніть лемматизацію та стемінг текстів у наборі даних. Порівняйте ефективність обох методів для виявлення сарказму. Використайте бібліотеку `nltk` для стемінгу через `PorterStemmer`, а для лемматизації скористайтеся `spacy` через функцію `nlp.pipe()` для пакетного оброблення.

---

**Варіант 3:** Виконайте токенізацію текстів та проаналізуйте частоту вживання окремих слів. Оцініть вплив високочастотних і низькочастотних слів на побудову моделей. Використайте `CountVectorizer` з бібліотеки `sklearn.feature_extraction.text` для обчислення кількості частоти слів.

---

**Варіант 4:** Проаналізуйте баланс класів у наборі даних. Оцініть наявність дисбалансу між позитивними та негативними прикладами сарказму. Скористайтесь функцією `value_counts()` у Pandas для виведення статистики кожного класу.

---

**Варіант 5:** Проаналізуйте, як зміна регістру тексту (великі/малі літери) може впливати на класифікацію сарказму. Виконайте нормалізацію регістру за допомогою методу `str.lower()` у Pandas і порівняйте результати до та після цієї обробки.

---

**Варіант 6:** Проаналізуйте та видаліть стоп-слова з тексту. Оцініть, як видалення стоп-слів впливає на побудову моделей. Використайте стандартні списки стоп-слів із бібліотек `nltk` (`nltk.corpus.stopwords`) або `sklearn.feature_extraction.text.TfidfVectorizer(stop_words='english')`.

---

**Варіант 7:** Виконайте аналіз біграм у текстах. Оцініть, які поєднання слів (біграми) найчастіше трапляються в саркастичних висловлюваннях. Використайте `TfidfVectorizer` з налаштуванням параметра `ngram_range=(2, 2)`.

---

**Варіант 8:** Проаналізуйте вживання числових значень у текстах. Оцініть, як їх наявність може впливати на класифікацію сарказму, і чи є різниця між текстами із числами та без них. Для цього скористайтесь регулярними виразами (`re.findall(r'\d+', text)`) для пошуку чисел.

---

**Варіант 9:** Проведіть аналіз частоти вживання знаків пунктуації, таких як крапки, коми, знаки оклику та питання. Визначте, як ці символи корелюють із наявністю сарказму в текстах. Для цього скористайтесь регулярними виразами (`re.findall(r'[?!.,]', text)`).

---

**Варіант 10:** Проаналізуйте частоту використання числових значень у текстах (наприклад, дати, години, числа). Оцініть, як їхня наявність може впливати на моделі виявлення сарказму, використовуючи регулярні вирази для пошуку чисел (`re.findall(r'\d+', text)`).

---

**Варіант 11:** Виконайте аналіз частоти появи та впливу цитат і прямих звернень у текстах на результати моделі. Для пошуку таких елементів використайте регулярні вирази (`re.findall(r'\"[^\"]+\"', text)`).

---

**Варіант 12:** Проведіть аналіз текстів за розміром шрифтів, наприклад, коли всі літери набрані великим регістром (CAPS LOCK). Використайте метод `str.isupper()` у Pandas для виявлення таких текстів і оцініть їх частоту в наборі даних.

---

**Варіант 13:** Оцініть вплив символів "?» і "!» на визначення сарказму. Виконайте підрахунок частоти їхньої появи за допомогою регулярних виразів і оцініть зв’язок із саркастичними повідомленнями, використовуючи `text.count('!')` або `text.count('?')`.

---

**Варіант 14:** Проаналізуйте та підготуйте набір даних для моделі, що враховує контекстні зв’язки між коментарями та відповідями. Використайте бібліотеку `nltk` для сегментації діалогів та побудуйте модель на основі діалогів, використовуючи відповідні контекстні репліки.

---

**Варіант 15:** Виконайте аналіз частоти використання абревіатур та скорочень у текстах. Оцініть, як часто вони з’являються в саркастичних висловлюваннях і впливають на продуктивність моделі. Для пошуку скорочень використайте регулярні вирази (`re.findall(r'\b[A-Z]{2,}\b', text)`).

---

**Варіант 16:** Проведіть аналіз частоти вживання запозичених слів або сленгу. Як ці елементи мови можуть впливати на розпізнавання сарказму? Для цього скористайтесь зовнішніми словниками або бібліотекою `nltk` для пошуку специфічних слів.

---

**Варіант 17:** Проаналізуйте наявність посилань на інші джерела (URLs) у текстах. Як часто сарказм супроводжується гіперпосиланнями і як це впливає на класифікацію? Використайте регулярний вираз для пошуку URL (`re.findall(r'http\S+', text)`).

---

**Варіант 18:** Виконайте попередній аналіз текстів на наявність повторів або копіювання певних фраз чи виразів. Оцініть, як це може вплинути на точність моделі виявлення сарказму, використовуючи функцію `duplicated()` у Pandas для пошуку дублікатів.

---

**Варіант 19:** Виконайте аналіз тональності текстів (позитивний, нейтральний, негативний). Оцініть вплив тональності на виявлення сарказму за допомогою моделей аналізу сентименту, таких як `TextBlob` або `VADER` з бібліотеки `nltk`.

---

**Варіант 20:** Проаналізуйте вплив багатомовності на виявлення сарказму в текстах. Чи є суттєві відмінності в саркастичних висловлюваннях різними мовами в змішаному наборі даних? Використайте бібліотеку `langdetect` для виявлення мови тексту.

<a class="anchor" id="lab-4.2"></a>

## <span style="color:blue; font-size:1.2em;">4.2. Векторизація TF-IDF та логістична регресія</span>

[Повернутися до змісту](#lab-4)

In [11]:
# Побудуємо біграми, обмежимо максимальну кількість ознак
# і мінімальну частоту слів
tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=5000, min_df=2)
# Створимо об'єкт мультиноміальної логістичної регресії,
# яка також відома як класифікатор softmax
logit = LogisticRegression(C=1, n_jobs=4, solver="lbfgs", random_state=17, verbose=1)
# Задамо sklearn's pipeline
tfidf_logit_pipeline = Pipeline([("tf_idf", tf_idf), ("logit", logit)])

In [12]:
%%time
tfidf_logit_pipeline.fit(train_texts, y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    7.5s finished


CPU times: total: 18.2 s
Wall time: 26.9 s


Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(max_features=5000, min_df=2,
                                 ngram_range=(1, 2))),
                ('logit',
                 LogisticRegression(C=1, n_jobs=4, random_state=17,
                                    verbose=1))])

In [13]:
%%time
valid_pred = tfidf_logit_pipeline.predict(valid_texts)

CPU times: total: 3.72 s
Wall time: 4.03 s


In [14]:
print(f"Точність за валідаційним набором даних: \
    {accuracy_score(y_valid, valid_pred): .4f}")

Точність за валідаційним набором даних:      0.6983


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import cProfile
import pstats

# Розбиття набору даних
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17)

In [16]:
# Функція для інкапсуляції виконання пайплайну
def run_pipeline(train_texts, y_train, valid_texts):
    # Налаштування TfidfVectorizer
    tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=20000, min_df=5)

    # Створення Деревa Рішень з обмеженою глибиною
    decision_tree = DecisionTreeClassifier(max_depth=10, random_state=17)

    # Налаштування пайплайну
    tfidf_tree_pipeline = Pipeline([("tf_idf", tf_idf), ("decision_tree", decision_tree)])

    # Навчання моделі
    tfidf_tree_pipeline.fit(train_texts, y_train)

    # Виконання прогнозів
    valid_pred = tfidf_tree_pipeline.predict(valid_texts)
    
    return valid_pred

In [17]:
# Профілювання виконання пайплайну
profiler = cProfile.Profile()
profiler.enable()

# Виконання пайплайну
valid_pred = run_pipeline(train_texts, y_train, valid_texts)
profiler.disable()

# Виведення результатів профілювання
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

         35353936 function calls (35353926 primitive calls) in 53.117 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   53.117   26.559 C:\Users\radiu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3469(run_code)
        2    0.000    0.000   53.117   26.559 {built-in method builtins.exec}
        1    0.215    0.215   53.117   53.117 C:\Users\radiu\AppData\Local\Temp\ipykernel_11996\2777130815.py:6(<module>)
        1    0.011    0.011   52.902   52.902 C:\Users\radiu\AppData\Local\Temp\ipykernel_11996\1472510363.py:2(run_pipeline)
        1    0.000    0.000   47.320   47.320 C:\Users\radiu\anaconda3\lib\site-packages\sklearn\pipeline.py:374(fit)
        1    0.000    0.000   24.216   24.216 C:\Users\radiu\anaconda3\lib\site-packages\sklearn\pipeline.py:336(_fit)
        1    0.000    0.000   24.215   24.215 C:\Users\radiu\anaconda3\lib\site-packages\joblib\memory.py:348(__call_

In [18]:
my_result = accuracy_score(y_valid, valid_pred)

my_result

0.5903662136813695

<span style="color:red; font-size:2em;">Завдання 2</span>

---

**Варіант 1:** Використайте триграми (3-грамні комбінації слів) у векторизаторі TF-IDF для побудови моделі класифікації. Триграми дають змогу вловити контекст, коли три слова розташовані поруч. Використайте `TfidfVectorizer(ngram_range=(3, 3))` для векторизації, а потім застосуйте логістичну регресію. Оцініть, чи триграми покращують точність проти уніграм або біграм.

*Технічна примітка:* Застосуйте метод `fit_transform()` для векторизації текстів, а потім використайте `LogisticRegression()` для побудови моделі.

---

**Варіант 2:** Використайте поєднання уніграм та біграм у TF-IDF векторизаторі для кращого охоплення локального контексту. Це допоможе врахувати не тільки поодинокі слова, але й поширені пари слів. Використайте `ngram_range=(1, 2)` у `TfidfVectorizer`, щоб включити й уніграми, і біграми, та побудуйте модель на основі отриманих ознак.

*Технічна примітка:* Використайте метод `fit_transform()` для підготовки ознак, а після цього `LogisticRegression()` для навчання моделі.

---

**Варіант 3:** Дослідіть вплив регуляризації L1 (Lasso) на модель логістичної регресії. Регуляризація L1 додає штраф за величину коефіцієнтів, тим самим надаючи змогу деяким ознакам мати нульові ваги, що призводить до менш складної моделі. Використайте `LogisticRegression(penalty='l1', solver='liblinear')` і порівняйте результати з базовою моделлю без регуляризації.

*Технічна примітка:* Застосуйте параметр `C` для керування ступенем регуляризації та використайте метод `fit()` для навчання моделі.

---

**Варіант 4:** Застосуйте регуляризацію L2 (Ridge) до моделі логістичної регресії. L2 регуляризація змушує всі коефіцієнти бути малими, але не нульовими, що дає змогу враховувати всі ознаки, але зменшує їхні ваги. Використайте `LogisticRegression(penalty='l2', solver='lbfgs')` для навчання моделі з регуляризацією.

*Технічна примітка:* Використайте метод `fit()` для навчання, і порівняйте значення точності з моделлю без регуляризації.

---

**Варіант 5:** Проведіть експерименти з параметром `max_features` у TF-IDF векторизаторі, обмежуючи кількість ознак до 1000. Цей параметр дає змогу контролювати розмір простору ознак, що може прискорити процес навчання. Використайте `TfidfVectorizer(max_features=1000)` для обмеження кількості ознак та перевірте, як це впливає на якість моделі.

*Технічна примітка:* Застосуйте метод `fit_transform()` для побудови матриці ознак і проаналізуйте результати за допомогою метрик точності, F1-міри та інших.

---

**Варіант 6:** Підвищіть кількість ознак у TF-IDF векторизаторі до 20,000, щоб врахувати рідкісні слова та специфічні терміни. Використайте `TfidfVectorizer(max_features=20000)` і перевірте, як збільшення кількості ознак впливає на продуктивність моделі. Проаналізуйте, чи покращуються показники моделі.

*Технічна примітка:* Використайте методи `fit_transform()` і `LogisticRegression()` для підготовки та навчання моделі, після чого оцініть результати.

---

**Варіант 7:** Змініть поріг ймовірності для прийняття рішень у логістичній регресії з 0.5 на 0.7 і порівняйте результати. Це дасть можливість зменшити кількість хибнопозитивних результатів, але може збільшити кількість хибнонегативних. Для цього використайте метод `predict_proba()` і порівняйте отримані результати з різними порогами.

*Технічна примітка:* Використайте `predict_proba()` для отримання ймовірностей і `np.where()` для зміни порогового значення.

---

**Варіант 8:** Проведіть експерименти з кількома порогами ймовірності (наприклад, 0.3, 0.5, 0.7) у логістичній регресії. Дослідіть, як зміна порогу впливає на показники моделі, такі як точність, чутливість і специфічність. Використайте `predict_proba()` для отримання ймовірностей і метод `precision_recall_curve()` з `sklearn.metrics` для аналізу впливу порогу.

*Технічна примітка:* Застосуйте функцію `precision_recall_curve()` для побудови графіків.

---

**Варіант 9:** Застосуйте 10-кратну перехресну перевірку (k-fold cross-validation) для моделі логістичної регресії, щоб оцінити її стабільність і узагальнювальну здатність. Використайте метод `cross_val_score()` з бібліотеки `sklearn.model_selection` і порівняйте середнє значення точності та стандартне відхилення.

*Технічна примітка:* Використайте параметр `cv=10` у методі `cross_val_score()` для 10-кратної перевірки.

---

**Варіант 10:** Порівняйте 5-кратну та 10-кратну перехресну перевірку для моделі логістичної регресії, щоб оцінити, який метод дає більш стабільні результати. Застосуйте метод `cross_val_score()` для обох конфігурацій та проаналізуйте отримані результати.

*Технічна примітка:* Використайте параметри `cv=5` та `cv=10` у методі `cross_val_score()` і порівняйте середні значення метрик.

---

**Варіант 11:** Проаналізуйте значення коефіцієнтів моделі логістичної регресії, щоб зрозуміти, які ознаки найбільше впливають на класифікацію. Використайте атрибут `coef_`, щоб вивести вагові коефіцієнти для кожної ознаки.

*Технічна примітка:* Після навчання моделі використайте атрибут `coef_` для аналізу значень коефіцієнтів.

---

**Варіант 12:** Порівняйте коефіцієнти моделі логістичної регресії для різних $n$-грам, щоб зрозуміти, як певні словосполучення впливають на класифікацію. Використайте атрибут `coef_` для отримання коефіцієнтів та побудуйте таблицю найбільш вагомих ознак.

*Технічна примітка:* Застосуйте функцію `get_feature_names_out()` для отримання назв ознак та `coef_` для їхніх коефіцієнтів.

---

**Варіант 13:** Побудуйте криву навчання для моделі логістичної регресії, щоб оцінити, як продуктивність моделі змінюється залежно від кількості навчальних даних. Використайте функцію `learning_curve()` з бібліотеки `sklearn.model_selection` для створення графіка.

*Технічна примітка:* Застосуйте параметри `train_sizes` для налаштування обсягів навчальних даних та `cv=5` для перехресної перевірки.

---

**Варіант 14:** Побудуйте криву валідації для логістичної регресії, щоби проаналізувати, як зміна параметра регуляризації впливає на продуктивність моделі. Використайте функцію `validation_curve()` з `sklearn.model_selection` і налаштуйте параметр `C`.

*Технічна примітка:* Використайте параметр `param_name='C'` у `validation_curve()` для оцінювання впливу регуляризації.

---

**Варіант 15:** Оптимізуйте параметри логістичної регресії за допомогою `GridSearchCV`. Налаштуйте параметри регуляризації (`C`) та тип регуляризації (`penalty`) для пошуку оптимальних значень. Застосуйте метод `GridSearchCV()` для налаштування та оцінювання продуктивності.

*Технічна примітка:* Використайте `param_grid` для налаштування діапазону значень параметрів і `scoring='accuracy'` для оцінювання продуктивності.

---

**Варіант 16:** Налаштуйте гіперпараметри логістичної регресії за допомогою `RandomizedSearchCV`, щоб оцінити, чи можна досягти таких самих або кращих результатів, як із `GridSearchCV`. Використайте більший діапазон значень для параметрів, щоб оптимізувати пошук.

*Технічна примітка:* Використайте параметр `n_iter=100` для обмеження кількості ітерацій і параметр `random_state` для відтворюваності.

---

**Варіант 17:** Виведіть назви ознак із TF-IDF векторизатора за допомогою функції `get_feature_names_out()` і проаналізуйте, які слова або фрази мають найбільший вплив на класифікацію. Порівняйте це з ваговими коефіцієнтами моделі логістичної регресії.

*Технічна примітка:* Використайте `get_feature_names_out()` для отримання ознак і поєднайте їх із коефіцієнтами за допомогою `coef_`.

---

**Варіант 18:** Побудуйте графік розподілу TF-IDF ваг для кожної ознаки за допомогою бібліотеки `matplotlib` або `seaborn`. Це допоможе візуалізувати, як часто певні слова з’являються в текстах та який вплив вони мають на модель.

*Технічна примітка:* Використайте метод `get_feature_names_out()` для отримання ознак і `coef_` для отримання ваг, а потім побудуйте графік за допомогою `seaborn.barplot()`.

---

**Варіант 19:** Проаналізуйте топ-20 найважливіших ознак, які мають найвищі TF-IDF ваги в моделі логістичної регресії. Використайте функцію `argsort()` для сортування коефіцієнтів і отримання найвпливовіших слів.

*Технічна примітка:* Застосуйте `argsort()` до `coef_`, щоб впорядкувати коефіцієнти за величиною та отримати топ-20 ознак.

---

**Варіант 20:** Проаналізуйте слова, що мають найменші TF-IDF значення, та дослідіть їхній вплив на модель. Розгляньте можливість вилучення таких ознак із моделі. Використайте `get_feature_names_out()` для отримання назв ознак та порівняйте їх із коефіцієнтами моделі.

*Технічна примітка:* Використайте функцію `argsort()` для пошуку найменших коефіцієнтів і їхнього аналізу.

<a class="anchor" id="lab-4.3"></a>

## <span style="color:blue; font-size:1.2em;">4.3. Інтерпретація та порівняння моделей</span>

[Повернутися до змісту](#lab-4)

In [19]:
# Довідково: Нижче подамо матрицю невідповідностей
def plot_confusion_matrix(
    actual,
    predicted,
    classes,
    normalize=False,
    title="Confusion matrix",
    figsize=(7, 7),
    cmap=plt.cm.Blues,
    path_to_save_fig=None,
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools

    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(actual, predicted).T
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("Predicted label")
    plt.xlabel("True label")

    if path_to_save_fig:
        plt.savefig(path_to_save_fig, dpi=300, bbox_inches="tight")

<span style="color:red; font-size:2em;">Завдання 3</span>

---

**Варіант 1:** Побудуйте криві навчання та валідування для моделі логістичної регресії, оцінюючи різницю між цими кривими на різних етапах навчання. Використайте функції `learning_curve()` з бібліотеки `sklearn.model_selection` та порівняйте отримані результати. Додатково побудуйте confusion matrix для оцінювання точності моделі, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте параметр `train_sizes` для поступового збільшення обсягу даних та оцініть можливе перенавчання чи недонавчання.

---

**Варіант 2:** Порівняйте криві навчання та валідування для логістичної регресії на основі використання різних значень параметра регуляризації (наприклад, `C`). Візуалізуйте результати за допомогою `validation_curve()` з `sklearn.model_selection` та побудуйте confusion matrix для аналізу результатів, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте діапазон значень параметра `C` у `validation_curve()` для виявлення найкращого балансу між перенавчанням та недонавчанням.

---

**Варіант 3:** Обчисліть та інтерпретуйте співвідношення шансів (odds ratios) для найбільш вагомих ознак у моделі логістичної регресії. Співвідношення шансів покаже, наскільки кожна ознака підвищує або знижує ймовірність сарказму. Побудуйте confusion matrix для обчислення точності моделі, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте атрибут `coef_` для отримання коефіцієнтів логістичної регресії та обчисліть odds ratios за допомогою функції `np.exp()`.

---

**Варіант 4:** Порівняйте співвідношення шансів для декількох важливих ознак у моделі логістичної регресії. Проаналізуйте, які ознаки найбільш впливають на класифікацію сарказму. Побудуйте confusion matrix для візуального подання помилок моделі, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте атрибут `coef_` для отримання коефіцієнтів і застосуйте формулу для обчислення odds ratios `exp(coef)`.

---

**Варіант 5:** Виконайте аналіз випадків, коли модель логістичної регресії дала неправильні прогнози. Використайте confusion matrix для виявлення таких випадків та візуалізуйте їх. На основі цього аналізу запропонуйте стратегії покращення моделі.

*Технічна примітка:* Використайте метод `predict()` для прогнозів та `plot_confusion_matrix()` для побудови матриці.

---

**Варіант 6:** Проаналізуйте помилки класифікації, що зроблені моделлю логістичної регресії, та побудуйте confusion matrix для візуалізації неправильно класифікованих випадків через самописну функцію `plot_confusion_matrix()`. Використайте цей аналіз для визначення проблемних ознак або класів та запропонуйте стратегії для покращення моделі.

*Технічна примітка:* Використайте `classification_report()` з `sklearn.metrics` для додаткового статистичного оцінювання моделі.

---

**Варіант 7:** Побудуйте модель дерева рішень за тими ж даними, що і для логістичної регресії, та порівняйте їхні результати. Для обох моделей побудуйте confusion matrix і проаналізуйте відмінності між результатами, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте `DecisionTreeClassifier` з `sklearn.tree`, а для логістичної регресії – `LogisticRegression`. Порівняйте precision, recall та F1-score.

---

**Варіант 8:** Порівняйте результативність моделі дерева рішень із логістичною регресією на основі confusion matrix. Проаналізуйте переваги та недоліки кожної моделі з огляду на їхні здатності класифікувати сарказм у текстах.

*Технічна примітка:* Використайте функцію `plot_confusion_matrix()` для кожної моделі та `GridSearchCV` для налаштування параметрів дерева рішень.

---

**Варіант 9:** Побудуйте графіки часткової залежності (PDP) для п’яти найважливіших ознак у моделі логістичної регресії, щоб оцінити, як зміна цих ознак впливає на ймовірність сарказму. Додатково побудуйте confusion matrix для оцінювання загальної продуктивності моделі, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте `plot_partial_dependence()` з `sklearn.inspection` для візуалізації часткової залежності.

---

**Варіант 10:** Створіть графіки часткової залежності для кількох найважливіших ознак у моделі логістичної регресії, щоб оцінити їхній вплив на класифікацію сарказму. Побудуйте confusion matrix для оцінювання точності моделі, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте параметр `features` у `plot_partial_dependence()` для вибору ключових ознак та візуалізації їхнього впливу.

---

**Варіант 11:** Оцініть модель логістичної регресії за допомогою метрик точності (accuracy), влучності (precision), повноти (recall) та F1-score. Для більш детального оцінювання побудуйте confusion matrix і проаналізуйте її, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте функції `accuracy_score()`, `precision_score()`, `recall_score()` та `f1_score()` з `sklearn.metrics` для обчислення метрик.

---

**Варіант 12:** Застосуйте кілька статистичних метрик, як от точність (accuracy), влучність (precision), повнота (recall) та F1-score, для оцінювання якості моделі логістичної регресії. Для візуалізації помилок скористайтесь confusion matrix, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте `classification_report()` для виведення всіх ключових метрик в одному місці та побудуйте на її основі confusion matrix.

---

**Варіант 13:** Проаналізуйте помилки моделі логістичної регресії, що були класифіковані неправильно. Використайте confusion matrix для ідентифікації цих випадків і порівняйте з правильно класифікованими.

*Технічна примітка:* Використайте функцію `plot_confusion_matrix()` для побудови матриці та знайдіть основні помилки моделі.

---

**Варіант 14:** Проведіть детальний аналіз випадків, коли модель логістичної регресії дала хибні прогнози. Візуалізуйте ці випадки за допомогою confusion matrix та проаналізуйте можливі причини помилок, використовуючи самописну функцію `plot_confusion_matrix()`.

*Технічна примітка:* Використайте метрики `precision`, `recall` та `f1_score()` для оцінювання продуктивності на окремих класах.

---

**Варіант 15:** Побудуйте та проаналізуйте криві навчання та валідування для моделі логістичної регресії, щоб оцінити її можливе перенавчання чи недонавчання. Використайте confusion matrix для виявлення класів, з якими модель найгірше справляється.

*Технічна примітка:* Використайте функції `learning_curve()` і `plot_confusion_matrix()` для візуалізації процесу навчання та його результатів.

---

**Варіант 16:** Застосуйте криві навчання та валідування до моделі логістичної регресії для оцінювання її стабільності. Використайте confusion matrix для обчислення точності моделі в різних класах на основі самописної функції `plot_confusion_matrix()`.

*Технічна примітка:* Параметр `cv=5` у `learning_curve()` допоможе побудувати криві для кращого оцінювання моделі.

---

**Варіант 17:** Проаналізуйте помилки класифікації в моделі логістичної регресії та візуалізуйте їх за допомогою confusion matrix. На основі аналізу запропонуйте стратегії для покращення точності класифікації та переобчисліть модель.

*Технічна примітка:* Використайте `GridSearchCV` для налаштування гіперпараметрів та повторно оцініть результати за допомогою `plot_confusion_matrix()`.

---

**Варіант 18:** Здійсніть детальний візуальний аналіз помилок моделі логістичної регресії, побудувавши confusion matrix на основі самописної функції `plot_confusion_matrix()`. Запропонуйте вдосконалення моделі за проведеним аналізом, а потім переобчисліть модель для покращення її точності.

*Технічна примітка:* Використайте метод `RandomizedSearchCV` для пошуку оптимальних гіперпараметрів.

---

**Варіант 19:** Створіть модель лінійної регресії, перетворивши цільову змінну для того, щоб вона стала кількісною. Побудуйте confusion matrix для порівняння результатів лінійної та логістичної регресій, використовуючи самописну функцію `plot_confusion_matrix()`. Оцініть, чому лінійна регресія може бути менш придатною для цього завдання.

*Технічна примітка:* Використайте функцію `LinearRegression()` з `sklearn.linear_model` та порівняйте результати з логістичною регресією.

---

**Варіант 20:** Змініть цільову змінну для моделі лінійної регресії та порівняйте її результати з логістичною регресією. Візуалізуйте помилки моделі за допомогою confusion matrix та зробіть висновки щодо доцільності використання лінійної регресії для класифікаційних задач.

*Технічна примітка:* Використайте `LinearRegression()` для побудови моделі та `plot_confusion_matrix()` для порівняння результатів.

<a class="anchor" id="lab-4.4"></a>

## <span style="color:blue; font-size:1.2em;">4.4. Вдосконалення моделі логістичної регресії</span>

[Повернутися до змісту](#lab-4)

<span style="color:red; font-size:2em;">Завдання 4</span>

---

**Варіант 1:** Розробіть два окремі TF-IDF векторизатори: один для слів, які є унікальними для кожного коментаря (що можуть вказувати на індивідуальний стиль автора), і другий – для загальних слів, які часто трапляються у всіх текстах. Унікальні слова можуть мати значення для виявлення сарказму, оскільки вони можуть відображати специфічні нюанси мови, тоді як загальні слова забезпечують контекст. Поєднайте ці два набори ознак за допомогою `FeatureUnion` для побудови загального набору ознак для моделі логістичної регресії та порівняйте результативність цієї моделі з базовою, використовуючи метрики precison, recall та F1-score.

*Технічна примітка:* Використайте два векторизатори `TfidfVectorizer()` з різними параметрами, наприклад, для уникнення загальних слів у одному векторизаторі та для їхнього включення в іншому (`max_df` або `min_df` для фільтрації). Для об’єднання ознак скористайтеся `FeatureUnion([('tfidf_unique', tfidf1), ('tfidf_common', tfidf2)])`. Для навчання моделі застосуйте `fit_transform()` для векторизації та порівняйте результативність за допомогою `classification_report()` з бібліотеки `sklearn.metrics`.

---

**Варіант 2:** Додайте ознаки, що відображають частоту використання великих літер (CAPS LOCK) у коментарях, оскільки саркастичні висловлювання часто супроводжуються підвищеним емоційним забарвленням, яке виражається через капіталізовані слова або фрази. Об’єднайте ці нові ознаки із TF-IDF векторизованими текстами та побудуйте модель логістичної регресії. Перевірте результативність нової моделі проти попередньої, використовуючи precision, recall та F1-score.

*Технічна примітка:* Використайте регулярний вираз `re.findall(r'\b[A-Z]+\b', text)` для обчислення кількості слів, що написані великими літерами в кожному коментарі. Цю інформацію додайте як нову ознаку до набору даних. Для побудови моделі скористайтесь `Pipeline()`, поєднавши TF-IDF векторизатор та логістичну регресію, і використайте `fit_transform()` для використання нових ознак.

---

**Варіант 3:** Інтегруйте аналіз тональності коментарів як нові ознаки до моделі логістичної регресії, щоб врахувати емоційне забарвлення тексту. Оцінювання полярності (позитивна, негативна або нейтральна) може бути особливо корисною під час класифікації сарказму, де емоційні індикатори часто вказують на приховані сенси. Використайте `TextBlob` для аналізу тональності текстів і додайте отримані значення як додаткові ознаки до набору TF-IDF ознак. Порівняйте результати з базовою моделлю та оцініть, як додаткові ознаки вплинули на загальну точність класифікації.

*Технічна примітка:* Використайте метод `TextBlob(text).sentiment.polarity` для оцінювання тональності кожного коментаря. Значення полярності змінюється від -1 (негативна тональність) до 1 (позитивна тональність). Додайте отримані оцінки як нові ознаки до DataFrame з основними TF-IDF ознаками та використайте `Pipeline()` для побудови та навчання моделі логістичної регресії.

---

**Варіант 4:** Побудуйте дві окремі моделі логістичної регресії для коротких і довгих коментарів, оскільки довжина тексту може впливати на точність виявлення сарказму. Короткі коментарі можуть містити більше концентрованого сарказму, тоді як у довших коментарях сарказм може бути замаскований складнішими виразами. Використайте середнє арифметичне ймовірностей або метод голосування (majority voting) для об’єднання результатів двох моделей і порівняйте їх із початковою моделлю. Оцініть, чи такий підхід покращує якість класифікації.

*Технічна примітка:* Використайте поріг, наприклад, 50 символів, для поділу коментарів на короткі та довгі. Навчіть дві окремі моделі логістичної регресії для кожної категорії. Для об’єднання результатів використайте `VotingClassifier()` з параметром `voting='soft'`, щоб об’єднати ймовірності з обох моделей, або параметр `voting='hard'`, якщо ви використовуєте majority voting. Порівняйте результати з базовою моделлю, використовуючи precision, recall та F1-score.

---

**Варіант 5:** Реалізуйте підхід, за яким треба додати частоти вживання певних ключових слів або словосполучень, які часто асоціюються із сарказмом (наприклад, «звісно», «безумовно», «о, так»). Ці ознаки можуть стати вагомими індикаторами сарказму в тексті, оскільки подібні фрази часто використовуються іронічно. Інтегруйте отримані частоти як нові ознаки в модель поряд із TF-IDF векторизованими даними. Оцініть, чи підвищується prevision, recall, та F1-score після додавання цих специфічних ознак.

*Технічна примітка:* Створіть окремі ознаки для обчислення кількості ключових слів у тексті за допомогою методу `str.count()`. Додайте ці ознаки до DataFrame з основними ознаками TF-IDF, після чого використайте `Pipeline()` для побудови та навчання моделі логістичної регресії.

---

**Варіант 6:** Розширте модель логістичної регресії, додавши ознаки, що подають часові метадані коментарів, як от день тижня, час публікації або частоту постів упродовж доби. Це може допомогти виявити тенденції, коли сарказм частіше вживається в певні періоди (наприклад, вечорами або на вихідних). Інтегруйте ці ознаки разом із TF-IDF векторизованими даними та порівняйте результативність моделі з попередньою версією.

*Технічна примітка:* Використайте метод `pd.to_datetime()` для перетворення дати публікації на часові ознаки, як от `hour`, `dayofweek`, або `month`. Ці ознаки додайте до DataFrame та інтегруйте з векторизованими текстовими даними.

---

**Варіант 7:** Замість традиційного підходу TF-IDF для векторизації тексту, спробуйте застосувати метод Word2Vec для векторизації слів, щоб захопити семантичний контекст і взаємозв’язок між словами. Word2Vec здатний враховувати латентні значення слів, що дає змогу краще розпізнавати сарказм у текстах. Побудуйте модель логістичної регресії на основі середнього значення векторів для кожного тексту та порівняйте її з моделлю, що використовує TF-IDF.

*Технічна примітка:* Використайте бібліотеку `gensim.models.Word2Vec` для навчання векторної моделі на текстах. Для кожного тексту обчисліть середнє значення векторів слів та використайте ці вектори як ознаки для логістичної регресії. Порівняйте результативність моделі за допомогою метрик precision, recall та F1-score.

---

**Варіант 8:** Використайте підхід до збільшення даних (data augmentation), щоб розширити набір даних, додаючи варіації оригінальних коментарів через заміну слів синонімами, перестановку частин речень або додавання нових слів. Це допоможе моделі навчитися краще розпізнавати сарказм за допомогою ширшого набору можливих текстових структур. Порівняйте точність моделі після додавання нових згенерованих даних із початковою моделлю, оцінюючи, чи збільшення даних покращило результати.

*Технічна примітка:* Використайте бібліотеку `nlaug` або `textaugment` для автоматичного збільшення даних, генеруючи додаткові варіанти коментарів. Інтегруйте нові коментарі до навчального набору даних і проведіть повторне навчання моделі логістичної регресії.

---

**Варіант 9:** Додайте ознаки, що подають розподіл частоти використання символів пунктуації (зокрема, знаків питання, оклику або лапок) у текстах. Пунктуація часто використовується для передачі сарказму, тому її аналіз може дати додаткові індикатори для класифікації. Інтегруйте ці ознаки разом із TF-IDF векторизованими даними та порівняйте, чи підвищується точність та інші метрики моделі.

*Технічна примітка:* Використайте метод `str.count()` для обчислення кількості частоти певних символів у текстах (наприклад, знаків питання `?`, оклику `!`, або лапок `"`). Додайте ці ознаки до DataFrame та використайте їх для навчання моделі логістичної регресії.

---

**Варіант 10:** Інтегруйте аналіз граматичних помилок у текстах як ознаку для покращення класифікації сарказму. Часто саркастичні коментарі містять навмисні або ненавмисні граматичні помилки, що може бути корисним для моделі. Додайте кількість граматичних помилок як додаткову ознаку та порівняйте результативність моделі до й після її додавання.

*Технічна примітка:* Використайте бібліотеку `language_tool_python` для автоматичного аналізу текстів на наявність граматичних помилок. Обчисліть кількість помилок у кожному тексті та додайте ці дані як нову ознаку до набору TF-IDF ознак.

---

**Варіант 11:** Додайте до моделі логістичної регресії ознаки, що відображають стиль написання коментарів (формальний або неформальний стиль). Це може включати частоту вживання скорочень (наприклад, «щас» замість «зараз») та сленгових виразів (наприклад, «лол», «кек», «збс» тощо). Такі ознаки можуть вказувати на неформальний стиль, який іноді пов’язаний із сарказмом. Обчисліть кількість цих елементів для кожного коментаря, щоб модель могла краще розрізняти різні стилі мовлення.

*Технічна примітка:* Створіть словники неформальних виразів, скорочень та сленгу, використовуючи, наприклад, API `UrbanDictionary` або створіть вручну словники для формального/неформального стилю. Підраховуйте частоту вживання цих елементів у кожному коментарі за допомогою методу `str.count()` та інтегруйте ці ознаки у DataFrame з основними TF-IDF ознаками.

---

**Варіант 12:** Додайте ознаки, що відображають риторику заперечення в коментарях (наприклад, частоту вживання заперечних конструкцій або слів типу «не», «ніколи», «жоден»). Сарказм часто використовується для підсилення заперечних висловлювань, де мовник іронічно відкидає певні факти або ситуації. Такі риторичні конструкції можуть бути корисними для розпізнавання саркастичних висловлювань. Інтегруйте ці ознаки в модель логістичної регресії та порівняйте її результати з базовою моделлю.

*Технічна примітка:* Використайте регулярні вирази для обчислення кількості заперечень у кожному коментарі, наприклад, `re.findall(r'\b(не|ніколи|жоден|ані)\b', text)`. Ці значення можуть бути збережені як нові ознаки в DataFrame. Об’єднайте їх із TF-IDF ознаками та побудуйте модель логістичної регресії за допомогою `Pipeline()`. Оцініть результативність нової моделі, порівнюючи precision, recall, F1-score та accuracy з базовою версією моделі.

---

**Варіант 13:** Побудуйте ансамблеву модель, що поєднує логістичну регресію та дерева рішень за допомогою методики стакінгу (stacking). Метод стакінгу дає змогу використовувати результати кількох моделей для покращення прогнозування внаслідок навчання «метамоделі», яка робить фінальне рішення на основі прогнозів базових моделей. Порівняйте результати цієї моделі з початковою логістичною регресією за допомогою метрик accuracy, precision, recall і F1-score.

*Технічна примітка:* Використайте `StackingClassifier()` із `sklearn.ensemble`, поєднуючи логістичну регресію та дерево рішень, або інші базові моделі, як-от SVM або Naive Bayes. Використайте `LogisticRegression` як метамодель, яка приймає рішення на основі виходів базових моделей. Оцініть точність та інші метрики з допомогою `cross_val_score()` або `classification_report()`.

---

**Варіант 14:** Використайте метод ансамблю підсумовування (bagging) на основі моделі логістичної регресії для покращення її стабільності та точності. Bagging допомагає зменшити варіативність і покращити стійкість моделі до вибіркового шуму, генеруючи кілька підвибірок даних для навчання кожної моделі в ансамблі. Порівняйте результативність моделі bagging з початковою логістичною регресією.

*Технічна примітка:* Використайте `BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=50, random_state=42)` з бібліотеки `sklearn.ensemble` для побудови ансамблю моделей. Порівняйте результати за допомогою метрик точності та F1-score за валідаційним набором даних, використовуючи метод `cross_val_score()`.

---

**Варіант 15:** Спробуйте застосувати кластеризацію для попереднього оброблення текстів, щоб групувати схожі коментарі перед їхньою векторизацією. Це може допомогти виявити патерни сарказму у великих текстових масивах і зменшити шум у даних. Додайте до моделі номер кластера як додаткову ознаку для кожного коментаря і порівняйте результативність цієї моделі з початковою логістичною регресією.

*Технічна примітка:* Використайте алгоритм `KMeans` із бібліотеки `sklearn.cluster` для кластеризації текстів на основі їхніх векторизованих відображень (наприклад, TF-IDF). Додайте номер кластера як нову ознаку до кожного коментаря та інтегруйте це в модель логістичної регресії. Оцініть результативність моделі за допомогою метрик precision, recall і F1-score, використовуючи метод `classification_report()`.

---

**Варіант 16:** Розширте модель логістичної регресії, додавши ознаки на основі синтаксичного аналізу текстів. Ці ознаки можуть включати частоту вживання різних частин мови, таких як дієслова, прикметники, прислівники та сполучники, які можуть вказувати на стилістичні або емоційні особливості сарказму. Синтаксичні структури можуть бути корисними для виявлення тонких натяків, властивих саркастичним висловлюванням. Порівняйте результативність цієї моделі з базовою.

*Технічна примітка:* Використайте бібліотеку `spacy` для синтаксичного аналізу. За допомогою атрибута `pos_` можна визначити частини мови кожного слова в тексті. Обчисліть кількість кожної частини мови та додайте ці дані як нові ознаки до DataFrame.

---

**Варіант 17:** Проаналізуйте контекст дискусій, враховуючи послідовність кількох коментарів, що йдуть підряд. Для сарказму важливим може бути контекст, у якому даний коментар опублікований, наприклад, якщо це відповідь на попередній коментар або репліка в межах певної теми. Додайте інформацію про відносне розташування коментаря в діалозі, його відповідність попереднім реплікам або реакцію на них, і порівняйте результати моделі.

*Технічна примітка:* Зберігайте попередні коментарі в межах одного діалогу та їхній зв’язок із поточним як нові ознаки (наприклад, `prev_comment_similarity`). Використайте текстову подібність, застосовуючи методи на основі векторів, як от cosine similarity з `sklearn.metrics.pairwise`.

---

**Варіант 18:** Додайте до моделі логістичної регресії ознаки, що відображають стиль написання тексту. Це може включати довжину речень, кількість складних синтаксичних структур або співвідношення довгих та коротких слів. Такі ознаки можуть бути індикаторами формальності або стилю, який іноді пов’язаний із сарказмом. Порівняйте результативність цієї моделі з базовою, оцінюючи такі метрики, як precision, recall та F1-score.

*Технічна примітка:* Використайте бібліотеку `spacy` для аналізу синтаксичних дерев тексту. Атрибут `sent_len` може використовуватись для обчислення кількості слів у реченнях, а `dependency_tree` – для оцінювання складності речень. Додайте ці ознаки до DataFrame і проаналізуйте їхній вплив на модель.

---

**Варіант 19:** Використайте додаткові ознаки, що ґрунтуються на тональності кожного окремого слова в тексті. Аналіз на рівні слів дасть можливість оцінювати емоційне забарвлення кожного слова, що може допомогти краще ідентифікувати сарказм, оскільки саркастичні висловлювання часто містять контрастні емоційні індикатори. Додайте ці оцінки як нові ознаки та порівняйте результативність моделі із базовою.

*Технічна примітка:* Використайте бібліотеку `VADER` з пакету `nltk` для визначення тональності на рівні кожного слова (positive, negative, neutral). Збережіть ці значення як нові ознаки та додайте їх до DataFrame для навчання моделі логістичної регресії.

---

**Варіант 20:** Реалізуйте вибіркове оброблення текстів, вибираючи лише певні типи речень, які найчастіше містять сарказм. Наприклад, зверніть увагу на речення, що починаються зі словосполучень типу «Ну, звісно», «О, точно», «Якби ж то», які часто використовуються для передачі іронії чи сарказму. Інтегруйте ці нові ознаки в модель і порівняйте її з базовою.

*Технічна примітка:* Використайте регулярні вирази (`re`) для знаходження ключових словосполучень на початку речень (наприклад, `r'^\b(Ну, звісно|О, точно)\b'`) і збережіть ці речення як окремі ознаки для кожного тексту. Додайте ці ознаки до основних даних для навчання моделі.

<a class="anchor" id="lab-4.5"></a>

## <span style="color:blue; font-size:1.2em;">4.5. Практичне застосування результатів інтелектуального аналізу даних</span>

[Повернутися до змісту](#lab-4)

<span style="color:red; font-size:2em;">Завдання 5</span>

---

**Варіант 1:** Застосуйте модель логістичної регресії до класифікації саркастичних і не саркастичних коментарів із набору даних *train-balanced-sarcasm.csv*, використовуючи всі ознаки, що були створені на попередніх етапах (емоційне забарвлення, заперечення, соціальні сигнали, частоти великих літер тощо). Проведіть фінальне навчання на цьому збалансованому наборі та оцініть результативність моделі на тестовому наборі даних. Порівняйте результати з базовою моделлю, що використовувала лише TF-IDF ознаки.

*Технічна примітка:* Використайте бібліотеку `scikit-learn` для розділення даних за допомогою `train_test_split()`. Навчіть модель за допомогою `LogisticRegression()` із налаштуванням регуляризації `C` для оптимізації. Порівняйте результати за допомогою `classification_report()` для метрик точності, precision, recall та F1-score.

---

**Варіант 2:** Проведіть аналіз результативності моделі логістичної регресії за незбалансованим набором даних, використовуючи всі створені ознаки (емоції, риторика, частота вживання великих літер). Проаналізуйте, як модель поводиться у випадках із великою кількістю негативних прикладів сарказму або недостатньою кількістю позитивних. Оцініть, чи потрібно застосовувати підходи до підвищення ваги позитивних класів для покращення результатів.

*Технічна примітка:* Використайте параметр `class_weight='balanced'` у моделі `LogisticRegression()` для автоматичного підвищення ваг класів. Додатково можете використати методи підвищення вибірки (oversampling) за допомогою `SMOTE` з бібліотеки `imblearn`.

---

**Варіант 3:** Створіть функцію для автоматичного прогнозування сарказму в нових коментарях, використовуючи модель логістичної регресії, навченої за набором даних *train-balanced-sarcasm.csv*. Додайте можливість прогнозування ймовірності для кожного коментаря та пояснення, які ознаки найбільше вплинули на класифікацію. Проаналізуйте важливість ознак для кожного передбаченого класу (сарказм/не сарказм).

*Технічна примітка:* Використайте метод `predict_proba()` з бібліотеки `scikit-learn` для отримання ймовірностей для кожного класу. Для пояснення важливості ознак використайте бібліотеки `LIME` або `SHAP`, щоб визначити, які саме ознаки найбільше впливають на результат.

---

**Варіант 4:** Застосуйте модель логістичної регресії до аналізу довгих і коротких коментарів у наборі *train-balanced-sarcasm.csv*. Проведіть аналіз, чи сарказм частіше трапляється в коротких коментарях або в довших текстах. Побудуйте дві окремі моделі для коротких та довгих коментарів і порівняйте їхні результати за точністю та F1-score.

*Технічна примітка:* Використайте функцію для поділу коментарів за довжиною (наприклад, коментарі, коротші за 50 символів – короткі, довші – довгі). Навчіть дві окремі моделі за допомогою `LogisticRegression()` для кожної категорії. Порівняйте результативність за допомогою метрик, таких як precision, recall та F1-score.

---

**Варіант 5:** Створіть модель, яка прогнозує наявність сарказму залежно від дня тижня або часу публікації коментарів, використовуючи метадані з *train-balanced-sarcasm.csv*. Проведіть аналіз, чи є залежність між частотою саркастичних коментарів та певними днями тижня (наприклад, чи частіше сарказм трапляється в п’ятницю ввечері). Використайте цю модель для прогнозування сарказму на основі тимчасових даних.

*Технічна примітка:* Використайте функцію `pd.to_datetime()` для перетворення часових міток у дні тижня та години. Додайте ці ознаки до основного набору TF-IDF ознак і побудуйте модель за допомогою `LogisticRegression()`. Проведіть аналіз точності моделі для кожного дня тижня, використовуючи `cross_val_score()` для отримання середніх показників результативності.

---

**Варіант 6:** Застосуйте модель логістичної регресії до класифікації саркастичних коментарів на реальних даних із набору *train-balanced-sarcasm.csv*, включивши всі ознаки, розроблені на попередніх етапах (емоційне забарвлення, заперечення, соціальні сигнали, частота великих літер, риторичні конструкції). Проведіть фінальне тестування моделі на раніше невидимих даних і порівняйте результати з базовою моделлю, яка використовувала лише TF-IDF ознаки. Оцініть, наскільки ці нові ознаки підвищили точність класифікації сарказму.

*Технічна примітка:* Використайте `LogisticRegression()` з параметром `C` для налаштування регуляризації. Для оброблення та об’єднання всіх ознак використайте клас `FeatureUnion` з бібліотеки `sklearn`. Порівняйте результати за допомогою метрик `precision`, `recall`, `F1-score`, використовуючи `classification_report()`.

---

**Варіант 7:** Проведіть експеримент із використанням моделі логістичної регресії на незбалансованому наборі даних із *train-balanced-sarcasm.csv*. Проаналізуйте вплив незбалансованості даних на результативність моделі, особливо в ситуаціях, де сарказм є менш представленим класом. Використайте метод підвищення ваг класів або збільшення кількості позитивних прикладів і порівняйте результати з попередніми.

*Технічна примітка:* Використайте параметр `class_weight='balanced'` у `LogisticRegression()` для автоматичного коригування ваг класів. Для збалансування набору даних також можете використати `SMOTE` з бібліотеки `imblearn`, що дасть змогу створити нові приклади позитивних класів.

---

**Варіант 8:** Розробіть функцію для класифікації нових коментарів як саркастичні чи ні, використовуючи модель логістичної регресії, навченої на *train-balanced-sarcasm.csv*. Додайте можливість прогнозування ймовірності кожного класу (сарказм/не сарказм) та збережіть результат разом з оцінкою ймовірності для подальшого аналізу точності моделі на нових даних.

*Технічна примітка:* Використайте функцію `predict_proba()` для виведення ймовірностей сарказму в коментарях. Збережіть результати прогнозування у DataFrame для подальшого аналізу та валідації точності моделі за новими метриками, такими як `precision`, `recall`, `F1-score`.

---

**Варіант 9:** Застосуйте модель логістичної регресії до аналізу темпоральної динаміки сарказму в коментарях. Використайте часові мітки з набору *train-balanced-sarcasm.csv*, щоб дослідити, чи є взаємозв’язок між сарказмом та часом публікації коментарів (наприклад, коментарі, опубліковані у вечірній час або в певні дні тижня). Порівняйте результати моделі, коли вона враховує або ігнорує часові ознаки.

*Технічна примітка:* Використайте `pd.to_datetime()` для перетворення часових міток у дні тижня або години. Додайте ці ознаки до загального набору ознак та використайте `LogisticRegression()` для оцінювання результативності моделі із часовими факторами та без них.

---

**Варіант 10:** Проведіть аналіз впливу довжини коментарів на результативність моделі логістичної регресії. Поділіть коментарі з *train-balanced-sarcasm.csv* на короткі та довгі (за заданим порогом, наприклад, 50 символів) і побудуйте окремі моделі для кожної категорії. Оцініть, чи сарказм частіше трапляється в коротких або довгих коментарях, і порівняйте результативність моделі для обох категорій.

*Технічна примітка:* Використайте метод для поділу коментарів за їхньою довжиною (наприклад, функцію `len()`). Побудуйте окремі моделі за допомогою `LogisticRegression()` для кожної групи та порівняйте результати за допомогою метрик точності (`accuracy`), precision, recall і F1-score.

---

**Варіант 11:** Застосуйте модель логістичної регресії до прогнозування сарказму в коментарях, з огляду на всі створені ознаки (емоційне забарвлення, заперечення, риторичні конструкції тощо). Проведіть фінальне тестування моделі, використовуючи крос-валідацію, та оцініть її точність на основі метрик точності, precision, recall та F1-score. Порівняйте результати цієї моделі з результатами базової моделі, що використовувала лише TF-IDF ознаки.

*Технічна примітка:* Використайте бібліотеку `scikit-learn`, зокрема клас `LogisticRegression()` для навчання моделі. для оцінювання результативності використайте функцію `cross_val_score()` з параметром `cv=10` для крос-валідації. Для порівняння результатів використайте `classification_report()` з метриками `accuracy`, `precision`, `recall` та `F1-score`.

---

**Варіант 12:** Створіть функцію для автоматичного прогнозування сарказму на нових даних, що не входять у тренувальний набір. Зробіть прогнозування та оцініть, як модель працює з новими даними. Додатково додайте аналіз важливості ознак, що найбільше впливають на результат, щоби покращити інтерпретацію моделі.

*Технічна примітка:* Використайте бібліотеку `SHAP` або `LIME` для інтерпретації впливу кожної ознаки на результат класифікації. Використайте методи векторизації та попередньо створені ознаки для аналізу нових даних для подальшого прогнозування моделі логістичної регресії.

---

**Варіант 13:** Проведіть аналіз результативності моделі логістичної регресії на різних підгрупах даних. Наприклад, розділіть коментарі на короткі та довгі (менше або більше, ніж 50 символів), або коментарі, опубліковані в різний час доби (ранок, день, вечір). Оцініть, як модель справляється з прогнозуванням сарказму в цих підгрупах, та порівняйте її точність для кожної з них.

*Технічна примітка:* Для розділення даних використайте функцію `len()` для підрахунку кількості символів у коментарях або функцію `pd.to_datetime()` для перетворення часових міток. Використайте модель `LogisticRegression()` для навчання окремих моделей для кожної групи, і порівняйте результати за допомогою `classification_report()`.

---

**Варіант 14:** Розробіть функцію для аналізу результативності моделі логістичної регресії з різними комбінаціями ознак (емоційне забарвлення, соціальні сигнали, частоти великих літер тощо). Створіть кілька варіантів моделей, кожна з яких використовує різні підмножини ознак, і порівняйте їх за точністю, precision, recall та F1-score. Зробіть висновки про те, які ознаки найбільше впливають на точність моделі.

*Технічна примітка:* Використайте клас `LogisticRegression()` для навчання декількох моделей, кожна з яких використовує різні набори ознак. Порівняйте результати кожної моделі за допомогою `cross_val_score()` або `classification_report()` для аналізу метрик точності, precision, recall та F1-score. Для побудови моделей із різними ознаками використайте `FeatureUnion` з бібліотеки `sklearn`.

---

**Варіант 15:** Застосуйте модель логістичної регресії до аналізу динаміки сарказму в певний період часу (наприклад, упродовж тижня або місяця). Проведіть аналіз, чи змінюється частота саркастичних коментарів у певні дні або в певні часові проміжки (наприклад, вечори вихідних). Використайте модель для прогнозування на основі тимчасових даних.

*Технічна примітка:* Використайте метод `pd.to_datetime()` для перетворення часових міток у дні тижня або години публікації коментарів. Додайте ці часові ознаки до набору TF-IDF та навчіть модель логістичної регресії на основі цих даних. Для аналізу результативності моделі використайте `cross_val_score()` для оцінювання точності.

---

**Варіант 16:** Застосуйте модель логістичної регресії до передбачення сарказму на основі всіх ознак, що були створені раніше (емоційне забарвлення, заперечення, соціальні сигнали тощо). Після навчання моделі на основі даних *train-balanced-sarcasm.csv*, реалізуйте підхід, що використовує методи крос-валідації для оцінювання надійності моделі. Проаналізуйте результати та порівняйте їх із базовою моделлю, яка використовувала лише TF-IDF ознаки.

*Технічна примітка:* Використайте `LogisticRegression()` з параметром `C` для регуляризації. Використайте `cross_val_score()` з `cv=10` для виконання крос-валідації та порівняння результатів моделі за метриками `accuracy`, `precision`, `recall` та `F1-score`.

---

**Варіант 17:** Проведіть аналіз результативності моделі логістичної регресії на невідомих раніше даних. Для цього підготуйте додаткові коментарі з інших джерел, аналогічних до тих, що містяться у *train-balanced-sarcasm.csv*, та проведіть прогнозування. Проаналізуйте результати класифікації та порівняйте точність моделі на нових даних із початковою.

*Технічна примітка:* Для отримання нових даних скористайтеся будь-якими доступними текстовими файлами, подібними за структурою до набору *train-balanced-sarcasm.csv*. Проведіть попередню обробку тексту (векторизацію) і застосуйте модель логістичної регресії до аналізу точності на нових даних. Порівняйте результати з оригінальними результатами моделі.

---

**Варіант 18:** Проведіть аналіз впливу різних типів ознак на результативність моделі логістичної регресії. Створіть кілька варіантів моделі: одну з використанням тільки емоційних ознак, іншу – із соціальними сигналами, а третю – з усіма ознаками разом. Оцініть, які саме типи ознак дають найбільший приріст точності моделі для класифікації сарказму.

*Технічна примітка:* Створіть окремі моделі, де використовуються лише певні типи ознак (наприклад, емоції через `TextBlob`, частота великих літер через регулярні вирази). Проведіть порівняльний аналіз результатів кожної моделі за допомогою `classification_report()` та метрик `precision`, `recall`, `F1-score`.

---

**Варіант 19:** Реалізуйте стратегію побудови ансамблевої моделі, що поєднує логістичну регресію з іншими моделями (наприклад, деревом рішень або SVM) для підвищення загальної точності класифікації сарказму. Об’єднайте прогнози кількох моделей через голосування (majority voting) або обчислення середнього ймовірностей, і порівняйте результативність із базовою моделлю логістичної регресії.

*Технічна примітка:* Використайте `VotingClassifier()` з параметром `voting='soft'` для комбінування результатів кількох моделей (наприклад, `LogisticRegression()`, `DecisionTreeClassifier()`, `SVC()`). Порівняйте результативність ансамблевої моделі за допомогою метрик precision, recall та F1-score.

---

**Варіант 20:** Реалізуйте стратегію для автоматичного виявлення сарказму в коментарях у режимі офлайн, застосовуючи модель логістичної регресії до великого набору даних коментарів. Проаналізуйте результати прогнозування, зокрема виведіть середній відсоток саркастичних коментарів у наборі даних і порівняйте вплив різних ознак на точність прогнозів.

*Технічна примітка:* Використайте модель логістичної регресії для масового оброблення коментарів, зберігаючи результати класифікації у DataFrame. Для аналізу впливу ознак на класифікацію скористайтеся методами `coef_` для логістичної регресії або використайте бібліотеку `SHAP` для глибшого аналізу.